In [1]:
!pip install pandas numpy matplotlib scikit-learn scipy torch 

  Using cached pandas-2.2.3-cp312-cp312-macosx_11_0_arm64.whl.metadata (89 kB)
  Using cached numpy-2.2.2-cp312-cp312-macosx_14_0_arm64.whl.metadata (62 kB)
  Using cached matplotlib-3.10.0-cp312-cp312-macosx_11_0_arm64.whl.metadata (11 kB)
  Using cached scikit_learn-1.6.1-cp312-cp312-macosx_12_0_arm64.whl.metadata (31 kB)
  Using cached scipy-1.15.1-cp312-cp312-macosx_14_0_arm64.whl.metadata (61 kB)
  Using cached torch-2.6.0-cp312-none-macosx_11_0_arm64.whl.metadata (28 kB)
  Using cached pytz-2025.1-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.1-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached contourpy-1.3.1-cp312-cp312-macosx_11_0_arm64.whl.metadata (5.4 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached fonttools-4.56.0-cp312-cp312-macosx_10_13_universal2.whl.metadata (101 kB)
  Using cached kiwisolver-1.4.8-cp312-cp312-macosx_11_0_arm64.whl.metadata (6.2 kB)
  Using cached pillow-11.1.0-cp312-cp312-macosx_11_0_arm64.whl.m

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import os
from itertools import product
from sklearn.utils import resample
from scipy.stats import beta as bt
import torch
import torch.optim as optim
import torch.nn as nn
from scipy.optimize import minimize
import math
import ast

# Parameter fitting

In [508]:
df = pd.read_csv("regret_preregistered_1_for_sim.csv")
df.head()

,Unnamed: 0,workerid,trial_index,points,choice,choice_goodbad,condition,source,final_points,expected_value,total_expected_value
0,1,860,1,1,1,best,regret_late,data,31,0.7,35.4
1,2,860,2,1,1,best,regret_late,data,31,0.7,35.4
2,3,860,3,1,1,best,regret_late,data,31,0.7,35.4
3,4,860,4,1,1,best,regret_late,data,31,0.7,35.4
4,5,860,5,1,1,best,regret_late,data,31,0.7,35.4


In [543]:
df = df[
        ((df['condition'] == 'regret_early') & (df['trial_index'] <= 30)) | 
        ((df['condition'] == 'regret_late') & (df['trial_index'] > 30)) |
        (df['condition'] == 'control')]

df.head()


,Unnamed: 0,workerid,trial_index,points,choice,choice_goodbad,condition,source,final_points,expected_value,total_expected_value
30,31,860,31,1,1,best,regret_late,data,31,0.7,35.4
31,32,860,32,1,1,best,regret_late,data,31,0.7,35.4
32,33,860,33,0,1,best,regret_late,data,31,0.7,35.4
33,34,860,34,1,1,best,regret_late,data,31,0.7,35.4
34,35,860,35,1,0,medium,regret_late,data,31,0.5,35.4


## GiG

In [667]:

# -----------------------------------------------------------------------------
# Model definition
# -----------------------------------------------------------------------------
class RegretModel(nn.Module):
    def __init__(self, condition, initial_omega=0.5, initial_tau=0.5, kappa=1.0):
        """
        If condition is 'control', omega is fixed at 0 and is not learned.
        For regret conditions ('regret_early' or 'regret_late'), omega is learnable.
        
        kappa is a fixed scaling factor used to decouple the magnitude of the omega
        from the tau.
        """
        super().__init__()
        self.condition = condition  # store the participant's condition
        self.softplus = nn.Softplus()
        self.kappa = kappa 
        
        if condition == "control":
            self.log_omega = None  # omega is fixed at 0 for control
        else:
            def softplus_inverse(x):
                return math.log(math.exp(x) - 1.0)
            init_log_omega = softplus_inverse(initial_omega)
            self.log_omega = nn.Parameter(torch.tensor(init_log_omega, dtype=torch.float32))
        
        def softplus_inverse(x):
            return math.log(math.exp(x) - 1.0)
        init_log_tau = softplus_inverse(initial_tau)
        self.log_tau = nn.Parameter(torch.tensor(init_log_tau, dtype=torch.float32))
    
    def forward(self):
        if self.condition == "control":
            omega = torch.tensor(0.0, dtype=torch.float32)
        else:
            omega = self.softplus(self.log_omega) 
        tau = self.softplus(self.log_tau) 
        return omega, tau
    
    def log_likelihood(self, participant_df):
        """
        Computes the negative log likelihood for a single participant's data.
        """
        omega, tau = self.forward()
        num_arms = 3
        
        # Initialize belief parameters for each arm (alpha and beta for Beta distributions)
        alpha = torch.ones(num_arms, dtype=torch.float32)
        #decay_factor = 1.0  
        #alpha = alpha * decay_factor  
        beta  = torch.ones(num_arms, dtype=torch.float32)
        
        df_sorted = participant_df.sort_values('trial_index')
        nll = torch.tensor(0.0, dtype=torch.float32)
        
        for _, row in df_sorted.iterrows():
            choice      = int(row['choice'])      # index of the chosen arm
            points      = int(row['points'])      # outcome: 1 (reward) or 0 (no reward)
            trial_index = int(row['trial_index'])   # trial number
            
            p_est = alpha / (alpha + beta)
            numerators = torch.exp(p_est / tau)
            probs = numerators / torch.sum(numerators)
            
            p_chosen = probs[choice]
            nll -= torch.log(p_chosen + 1e-12)  # add to negative log likelihood for numerical stability
            
            # Update beliefs for the chosen arm based on actual outcome
            if points == 1:
                alpha = alpha.clone()
                alpha[choice] += 1.0
            else:
                beta = beta.clone()
                beta[choice] += 1.0
                
                # For regret conditions, apply counterfactual updating to the non-chosen arms
                if self.condition == 'regret_early' and trial_index <= 30 and points == 0:
                    alpha = alpha.clone()  # avoid in-place modifications
                    for arm_i in range(num_arms):
                        if arm_i != choice:
                            alpha[arm_i] += self.kappa * omega
                elif self.condition == 'regret_late' and trial_index > 30 and points == 0:
                    alpha = alpha.clone()
                    for arm_i in range(num_arms):
                        if arm_i != choice:
                            alpha[arm_i] += self.kappa * omega
        
        return nll

# -----------------------------------------------------------------------------
# Training loop for each participant
# -----------------------------------------------------------------------------

unique_participants = df['workerid'].unique()

# Hyperparameters
num_epochs    = 500
learning_rate = 0.01

print_interval = 100 

all_results = [] 

for participant in unique_participants:
    participant_df = df[df['workerid'] == participant]
    participant_condition = participant_df['condition'].iloc[0]
    
    model = RegretModel(condition=participant_condition, initial_omega=0.5, initial_tau=0.5, kappa=1.0)
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    
    # Training loop.
    for epoch in range(num_epochs):
        optimizer.zero_grad()
        nll = model.log_likelihood(participant_df)
        nll.backward()
        optimizer.step()
        
        if (epoch + 1) % print_interval == 0 or epoch == 0:
            with torch.no_grad():
                current_omega, current_tau = model.forward()
                print(f"Participant {participant}, Epoch {epoch+1}, NLL={nll.item():.4f}, "
                      f"omega={current_omega.item():.4f}, tau={current_tau.item():.4f}, condition={participant_condition}")
    
    with torch.no_grad():
        final_omega, final_tau = model.forward()
        final_nll = model.log_likelihood(participant_df).item()
    
    all_results.append({
        'participant': participant,
        'condition': participant_condition,
        'omega': final_omega.item(),  # check that it is 0 for control
        'tau': final_tau.item(),
        'nll': final_nll
    })

results_df = pd.DataFrame(all_results)

Participant 860, Epoch 1, NLL=29.8333, omega=0.4961, tau=0.4961, condition=regret_late
Participant 860, Epoch 100, NLL=27.6530, omega=0.2428, tau=0.2269, condition=regret_late
Participant 860, Epoch 200, NLL=27.6140, omega=0.2314, tau=0.2090, condition=regret_late
Participant 860, Epoch 300, NLL=27.6138, omega=0.2348, tau=0.2082, condition=regret_late
Participant 860, Epoch 400, NLL=27.6137, omega=0.2383, tau=0.2074, condition=regret_late
Participant 860, Epoch 500, NLL=27.6137, omega=0.2414, tau=0.2068, condition=regret_late
Participant 835, Epoch 1, NLL=30.3529, omega=0.4961, tau=0.4961, condition=regret_late
Participant 835, Epoch 100, NLL=18.7586, omega=0.1949, tau=0.1691, condition=regret_late
Participant 835, Epoch 200, NLL=4.0251, omega=0.0874, tau=0.0528, condition=regret_late
Participant 835, Epoch 300, NLL=2.9211, omega=0.0682, tau=0.0374, condition=regret_late
Participant 835, Epoch 400, NLL=2.7247, omega=0.0578, tau=0.0326, condition=regret_late
Participant 835, Epoch 500, 

In [668]:
results_df = pd.DataFrame(all_results)
results_df.to_csv('regret_1_sim.csv', index=False)

In [669]:
print(results_df.groupby("condition")["omega"].describe())

              count      mean       std       min       25%       50%  \
condition                                                               
control        56.0  0.000000  0.000000  0.000000  0.000000  0.000000   
regret_early   60.0  0.961966  1.414525  0.009352  0.045548  0.171793   
regret_late    59.0  0.551960  1.096354  0.007603  0.029389  0.062883   

                   75%       max  
condition                         
control       0.000000  0.000000  
regret_early  1.229877  4.880089  
regret_late   0.188459  4.646651  


In [670]:
print(results_df.groupby("condition")["tau"].describe())

              count      mean       std       min       25%       50%  \
condition                                                               
control        56.0  0.250853  0.236838  0.016771  0.089015  0.164758   
regret_early   60.0  0.166594  0.223225  0.018540  0.071478  0.121080   
regret_late    59.0  0.283661  0.387755  0.019137  0.060485  0.112862   

                   75%       max  
condition                         
control       0.321055  1.255719  
regret_early  0.169163  1.393482  
regret_late   0.302521  1.575055  


## CS

In [655]:

class RegretModel(nn.Module):
    def __init__(self, condition, initial_omega=0.5, initial_tau=0.5, kappa=1.0):
        """
        - If condition is 'control', omega is fixed at 0 and is not learned.
        - For regret conditions ('regret_early' or 'regret_late'), omega is learnable.
        """
        super().__init__()
        self.condition = condition
        self.softplus = nn.Softplus()
        self.kappa = kappa 

        def softplus_inverse(x):
            return math.log(math.exp(x) - 1.0)

        if condition == "control":
            self.log_omega = nn.Parameter(torch.tensor(float("-inf"), dtype=torch.float32), requires_grad=False)  
        else:
            init_log_omega = softplus_inverse(initial_omega)
            self.log_omega = nn.Parameter(torch.tensor(init_log_omega, dtype=torch.float32))

        init_log_tau = softplus_inverse(initial_tau)
        self.log_tau = nn.Parameter(torch.tensor(init_log_tau, dtype=torch.float32))

    def forward(self):
        """ Returns omega and tau. """
        omega = self.softplus(self.log_omega) if self.condition != "control" else torch.tensor(0.0, dtype=torch.float32)
        tau = self.softplus(self.log_tau) 
        return omega, tau
    
    def log_likelihood(self, participant_df):
        """
        Computes the negative log likelihood for a single participant's data.
        """
        omega, tau = self.forward()
        num_arms = 3

        alpha = torch.ones(num_arms, dtype=torch.float32, requires_grad=True)
        beta = torch.ones(num_arms, dtype=torch.float32, requires_grad=True)

        df_sorted = participant_df.sort_values('trial_index')
        nll = torch.tensor(0.0, dtype=torch.float32, requires_grad=True)

        for _, row in df_sorted.iterrows():
            choice = int(row['choice'])
            points = int(row['points'])
            trial_index = int(row['trial_index'])

            p_est = alpha / (alpha + beta)
            numerators = torch.exp(p_est / tau)
            probs = numerators / torch.sum(numerators)
            p_chosen = probs[choice]
            nll = nll - torch.log(p_chosen + 1e-12)

            alpha = alpha.clone()
            beta = beta.clone()

            if points == 1:
                alpha[choice] += 1.0
            else:
                beta[choice] += 1.0

                if (self.condition == 'regret_early' and trial_index <= 30 and points == 0) or \
                   (self.condition == 'regret_late' and trial_index > 30 and points == 0):
                    
                    p_est_np = (alpha / (alpha + beta)).detach().cpu().numpy()
                    p_est_np = np.clip(p_est_np, 1e-6, 1 - 1e-6) 

                    alpha = alpha.clone()  
                    for arm_i in range(num_arms):
                        if arm_i != choice:
                            probs_vec = np.array([1 - p_est_np[arm_i], p_est_np[arm_i]])
                            probs_vec /= probs_vec.sum() 
                            counterfactual_outcome = np.random.choice([0, 1], p=probs_vec)

                            if counterfactual_outcome == 1:
                                alpha[arm_i] += self.kappa * omega
                            else:
                                beta[arm_i] += self.kappa * omega

        return nll

# -----------------------------------------------------------------------------
# Training loop for each participant
# -----------------------------------------------------------------------------
unique_participants = df['workerid'].unique()

num_epochs = 500
learning_rate = 0.01  

print_interval = 100

all_results = []

for participant in unique_participants:
    participant_df = df[df['workerid'] == participant]
    participant_condition = participant_df['condition'].iloc[0]

    model = RegretModel(condition=participant_condition, initial_omega=0.5, initial_tau=0.5, kappa=1.0)
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    for epoch in range(num_epochs):
        optimizer.zero_grad()
        nll = model.log_likelihood(participant_df)
        nll.backward()

        optimizer.step()

        if (epoch + 1) % print_interval == 0 or epoch == 0:
            with torch.no_grad():
                omega_val, tau_val = model.forward()
                print(f"Participant={participant}, Epoch={epoch+1}, "
                      f"NLL={nll.item():.4f}, "
                      f"omega={omega_val.item():.4f}, tau={tau_val.item():.4f}, condition={participant_condition}")

    final_omega, final_tau = model.forward()
    final_nll = model.log_likelihood(participant_df).item()

    all_results.append({
        'participant': participant,
        'condition': participant_condition,
        'omega': final_omega.item(),
        'tau': final_tau.item(),
        'nll': final_nll
    })

results_df = pd.DataFrame(all_results)

Participant=860, Epoch=1, NLL=29.6243, omega=0.4961, tau=0.4961, condition=regret_late
Participant=860, Epoch=100, NLL=27.7998, omega=0.2628, tau=0.2726, condition=regret_late
Participant=860, Epoch=200, NLL=28.2695, omega=0.1755, tau=0.2714, condition=regret_late
Participant=860, Epoch=300, NLL=27.7748, omega=0.1393, tau=0.2690, condition=regret_late
Participant=860, Epoch=400, NLL=27.3613, omega=0.1130, tau=0.2684, condition=regret_late
Participant=860, Epoch=500, NLL=27.6699, omega=0.0879, tau=0.2683, condition=regret_late
Participant=835, Epoch=1, NLL=22.1503, omega=0.5039, tau=0.4961, condition=regret_late
Participant=835, Epoch=100, NLL=9.8088, omega=0.3230, tau=0.1990, condition=regret_late
Participant=835, Epoch=200, NLL=3.1783, omega=0.1940, tau=0.0742, condition=regret_late
Participant=835, Epoch=300, NLL=3.9752, omega=0.1404, tau=0.0475, condition=regret_late
Participant=835, Epoch=400, NLL=2.6249, omega=0.1101, tau=0.0386, condition=regret_late
Participant=835, Epoch=500, N

In [661]:
results_df = pd.DataFrame(all_results)
results_df.to_csv('cf_1_sim.csv', index=False)

In [663]:
print(results_df.groupby("condition")["omega"].describe())

              count      mean       std       min       25%       50%  \
condition                                                               
control        56.0  0.000000  0.000000  0.000000  0.000000  0.000000   
regret_early   60.0  0.084729  0.047341  0.035466  0.060720  0.072450   
regret_late    60.0  0.114172  0.115842  0.031945  0.055109  0.082259   

                   75%       max  
condition                         
control       0.000000  0.000000  
regret_early  0.086465  0.254171  
regret_late   0.122989  0.717421  


In [662]:
print(results_df.groupby("condition")["tau"].describe())

              count      mean       std       min       25%       50%  \
condition                                                               
control        56.0  0.250853  0.236838  0.016771  0.089015  0.164758   
regret_early   60.0  0.279176  0.299500  0.024239  0.137052  0.180928   
regret_late    60.0  0.327517  0.434469  0.026520  0.069338  0.138669   

                   75%       max  
condition                         
control       0.321055  1.255719  
regret_early  0.295445  1.598133  
regret_late   0.320952  1.731511  


# Generating trial data from the model 

In [3]:

participants_df = pd.read_csv('regret_1_sim.csv')

participants_df['omega'] = participants_df.apply(lambda row: 0 if row['condition'] == 'control' else row['omega'], axis=1)

# exclude participant 797
participants_df = participants_df[participants_df['participant'] != 797]
participants_df = participants_df[participants_df['participant'] != 881]

participants_df.head()

,participant,condition,omega,tau,nll
0,860,regret_late,0.378480,0.357759,28.756001
1,835,regret_late,0.125176,0.209758,18.831045
2,753,regret_late,0.501065,0.690517,33.319092
3,833,regret_late,0.478296,0.584262,32.419643
4,841,regret_late,0.334865,0.992489,34.548622


## GiG

In [15]:

num_trees = 3
num_trials = 60 
tree_probs = [0.7, 0.5, 0.2] 

tree_probabilities = {i: prob for i, prob in enumerate(tree_probs)}

sorted_trees = sorted(tree_probabilities, key=tree_probabilities.get, reverse=True)
best_tree = sorted_trees[0]    # index of the best tree
medium_tree = sorted_trees[1]  # index of the medium tree
worst_tree = sorted_trees[2]   # index of the worst tree

all_data = []

# -----------------------
# Loop over each participant
# -----------------------
for idx, row in participants_df.iterrows():
    
    participant_id = row['participant']
    temperature = row['tau']            
    counterfactual_weight = row['omega'] 
    condition = row['condition']       
    
    num_simulations = 100
    
    for sim_id in range(num_simulations):
        tree_beliefs = np.ones(num_trees) / num_trees  # initial beliefs
        alpha = np.ones(num_trees)  # prior alpha for Beta distribution
        beta = np.ones(num_trees)   # prior beta for Beta distribution
        points = np.zeros(num_trials)
        
        # Run RL agent for the specified number of trials
        for trial in range(num_trials):
            exp_values = np.exp(tree_beliefs / temperature)
            softmax_values = exp_values / np.sum(exp_values)
            
            # select action based on the softmax probabilities
            action = np.random.choice(np.arange(num_trees), p=softmax_values)
            
            # simulate outcome of the chosen tree
            outcome = np.random.choice([0, 1], p=[1 - tree_probs[action], tree_probs[action]])
            points[trial] = outcome

            # GiG updating
            if (condition == 'regret_early' and trial <= 30):
                if outcome == 0:
                    for tree in range(num_trees):
                        if tree != action:
                            alpha[tree] += counterfactual_weight

            if (condition == 'regret_late' and trial > 30):
                if outcome == 0:
                    for tree in range(num_trees):
                        if tree != action:
                            alpha[tree] += counterfactual_weight
            
            # Update alpha/beta for the chosen tree based on the actual outcome
            if outcome == 1:
                alpha[action] += 1
            else:
                beta[action] += 1

            for tree in range(num_trees):
                successes = alpha[tree]
                failures = beta[tree]
                tree_beliefs[tree] = successes / (successes + failures)
            
            if action == best_tree:
                choice_goodbad = 'best'
            elif action == medium_tree:
                choice_goodbad = 'medium'
            else:
                choice_goodbad = 'worst'

            all_data.append({
                'participant': participant_id,
                'tau': temperature,
                'omega': row['omega'],
                'trial_index': trial,
                'points': points[trial],
                'choice_goodbad': choice_goodbad,
                'condition': f'regret_{condition}' 
            })

# -----------------------
# Convert to DataFrame and Save
# -----------------------
df_all = pd.DataFrame(all_data)

df_all = df_all[['participant', 'tau', 'omega', 'trial_index',
                 'points', 'choice_goodbad', 'condition']]

df_all.to_csv('simulation_output_all_participants.csv', index=False)

## Null

In [16]:

num_trees = 3
num_trials = 60  
tree_probs = [0.7, 0.5, 0.2]  

tree_probabilities = {i: prob for i, prob in enumerate(tree_probs)}

sorted_trees = sorted(tree_probabilities, key=tree_probabilities.get, reverse=True)
best_tree = sorted_trees[0]    # index of the best tree
medium_tree = sorted_trees[1]  # index of the medium tree
worst_tree = sorted_trees[2]   # index of the worst tree

all_data = []

# -----------------------
# Loop over each participant
# -----------------------
for idx, row in participants_df.iterrows():
    
    participant_id = row['participant']
    temperature = row['tau']
    condition = row['condition']    
    
    num_simulations = 100
    
    for sim_id in range(num_simulations):
        tree_beliefs = np.ones(num_trees) / num_trees  # initial beliefs
        alpha = np.ones(num_trees)  # prior alpha for Beta distribution
        beta = np.ones(num_trees)   # prior beta for Beta distribution
        points = np.zeros(num_trials)
        
        # Run RL agent for the specified number of trials
        for trial in range(num_trials):
            exp_values = np.exp(tree_beliefs / temperature)
            softmax_values = exp_values / np.sum(exp_values)
            
            # Select action based on the softmax probabilities
            action = np.random.choice(np.arange(num_trees), p=softmax_values)
            
            # Simulate outcome of the chosen tree
            outcome = np.random.choice([0, 1], p=[1 - tree_probs[action], tree_probs[action]])
            points[trial] = outcome
            
            # Update alpha/beta for the chosen tree based on the actual outcome
            if outcome == 1:
                alpha[action] += 1
            else:
                beta[action] += 1
            
            for tree in range(num_trees):
                successes = alpha[tree]
                failures = beta[tree]
                tree_beliefs[tree] = successes / (successes + failures)
            
            if action == best_tree:
                choice_goodbad = 'best'
            elif action == medium_tree:
                choice_goodbad = 'medium'
            else:
                choice_goodbad = 'worst'
            
            all_data.append({
                'participant': participant_id,
                'tau': temperature,
                'omega': row['omega'],
                'trial_index': trial,
                'points': points[trial],
                'choice_goodbad': choice_goodbad,
                'condition': condition  
            })
# -----------------------
# Convert to DataFrame and Save
# -----------------------
df_all = pd.DataFrame(all_data)

df_all = df_all[['participant', 'tau', 'omega', 'trial_index',
                 'points', 'choice_goodbad', 'condition']]

df_all.to_csv('simulation_output_all_participants_null.csv', index=False)

## CS

In [14]:

participants_df = pd.read_csv('cf_1_sim.csv')

participants_df['omega'] = participants_df.apply(lambda row: 0 if row['condition'] == 'control' else row['omega'], axis=1)

participants_df = participants_df[participants_df['participant'] != 797]
participants_df = participants_df[participants_df['participant'] != 881]

participants_df.head()

,participant,condition,omega,tau,nll
0,860,regret_late,0.241443,0.206786,27.613682
1,835,regret_late,0.049607,0.030124,2.631860
2,753,regret_late,4.646651,1.325840,32.893864
3,833,regret_late,0.132556,0.480181,32.321712
4,841,regret_late,0.032136,1.405803,32.961685


In [21]:

num_trees = 3
num_trials = 60 
tree_probs = [0.7, 0.5, 0.2] 

tree_probabilities = {i: prob for i, prob in enumerate(tree_probs)}

sorted_trees = sorted(tree_probabilities, key=tree_probabilities.get, reverse=True)
best_tree = sorted_trees[0]    # index of the best tree
medium_tree = sorted_trees[1]  # index of the medium tree
worst_tree = sorted_trees[2]   # index of the worst tree

all_data = []

# -----------------------
# Loop over each participant
# -----------------------
for idx, row in participants_df.iterrows():
    
    participant_id = row['participant']
    temperature = row['tau']    
    counterfactual_weight = row['omega'] 
    condition = row['condition']       
    
    num_simulations = 100
    
    for sim_id in range(num_simulations):
        tree_beliefs = np.ones(num_trees) / num_trees  # initial beliefs
        alpha = np.ones(num_trees)  # prior alpha for Beta distribution
        beta = np.ones(num_trees)   # prior beta for Beta distribution
        points = np.zeros(num_trials)
        
        # Run RL agent for the specified number of trials
        for trial in range(num_trials):
            exp_values = np.exp(tree_beliefs / temperature)
            softmax_values = exp_values / np.sum(exp_values)
            
            # Select action based on softmax probabilities
            action = np.random.choice(np.arange(num_trees), p=softmax_values)
            
            # Simulate outcome of the chosen tree
            outcome = np.random.choice([0, 1], p=[1 - tree_probs[action], tree_probs[action]])
            points[trial] = outcome
            
            # CS updating
            if condition == "regret_early" and trial <= 30 and outcome == 0:
                for tree in range(num_trees):
                    if tree != action:
                        # Initialize a counterfactual outcome vector and simulate for this tree based on beliefs
                        counterfactual_outcome = np.zeros(num_trees)
                        counterfactual_outcome[tree] = np.random.choice(
                            [0, 1], p=[1 - tree_beliefs[tree], tree_beliefs[tree]]
                        )
                        if counterfactual_outcome[tree] == 1:
                            alpha[tree] += counterfactual_weight
                        else:
                            beta[tree] += counterfactual_weight

            if condition == "regret_late" and trial > 30 and outcome == 0:
                for tree in range(num_trees):
                    if tree != action:
                        # Initialize a counterfactual outcome vector and simulate for this tree based on beliefs
                        counterfactual_outcome = np.zeros(num_trees)
                        counterfactual_outcome[tree] = np.random.choice(
                            [0, 1], p=[1 - tree_beliefs[tree], tree_beliefs[tree]]
                        )
                        if counterfactual_outcome[tree] == 1:
                            alpha[tree] += counterfactual_weight
                        else:
                            beta[tree] += counterfactual_weight
            
            # Update alpha/beta for the chosen tree based on the actual outcome
            if outcome == 1:
                alpha[action] += 1
            else:
                beta[action] += 1

            for tree in range(num_trees):
                successes = alpha[tree]
                failures = beta[tree]
                tree_beliefs[tree] = successes / (successes + failures)
            
            if action == best_tree:
                choice_goodbad = 'best'
            elif action == medium_tree:
                choice_goodbad = 'medium'
            else:
                choice_goodbad = 'worst'
            
            all_data.append({
                'participant': participant_id,
                'tau': temperature,
                'omega': row['omega'],
                'trial_index': trial,
                'points': points[trial],
                'choice_goodbad': choice_goodbad,
                'condition': f'regret_{condition}'
            })

# -----------------------
# Convert to DataFrame and Save
# -----------------------
df_all = pd.DataFrame(all_data)

df_all = df_all[['participant', 'tau', 'omega', 'trial_index',
                 'points', 'choice_goodbad', 'condition']]

df_all.to_csv('cf_simulation_output_all_participants.csv', index=False)